<img title="GitHub Octocat" src='./img/Octocat.jpg' style='height: 60px; padding-right: 15px' alt="Octocat" align="left"> This notebook is part of a GitHub repository: https://github.com/pessini/moby-bikes 
<br>MIT Licensed
<br>Author: Leandro Pessini

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import sys
import os
import joblib

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Models & Evaluation
from sklearn.model_selection import KFold

# statsmodel
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf
import statsmodels.stats as stats

# Boost models
import xgboost as xgb
from sklearn import metrics

# Hyperparameter optimization
from bayes_opt import BayesianOptimization

# Custom objects
sys.path.insert(0, os.path.abspath('../src/'))
import experiment_tracker as et

import time
import warnings
warnings.simplefilter('ignore', FutureWarning)
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [2]:
# ideas_df = pd.read_excel('../documentation/experiment_tracker.xlsx', sheet_name='Ideas')
# experiments_df = pd.read_excel('../documentation/experiment_tracker.xlsx', sheet_name='Experiments')

In [3]:
# creates a new object to keep track of the experiments
experiment_tracker = et.ExperimentTracker()

In [4]:
df_train = pd.read_csv('../data/processed/df_train.csv')
df_test = pd.read_csv('../data/processed/df_test.csv')

In [5]:
df = df_train.copy()
X = df.drop(['count'], axis=1)
y = df.pop('count')
all_columns = list(X.columns)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
X.shape

(8760, 22)

In [6]:
test_df = df_test.copy()
X_test = test_df.drop(['count'], axis=1)
y_test = test_df.pop('count')
X_test.shape

(1464, 22)

In [7]:
def get_metrics_to_Experiment(dict_scores = None) -> list:
    if dict_scores is None:
        dict_scores = {}
    rsme = et.Score('RSME', '{:.4f}'.format(dict_scores['train_rsme']), '{:.4f}'.format(dict_scores['val_rsme']))
    mae = et.Score('MAE', '{:.4f}'.format(dict_scores['train_mae']), '{:.4f}'.format(dict_scores['val_mae']))
    return [rsme, mae]

In [8]:
import category_encoders as ce

def preprocessor(predictors: list) -> ColumnTransformer:
    # Setting remainder='passthrough' will mean that all columns not specified in the list of “transformers” 
    #   will be passed through without transformation, instead of being dropped

    ##################### Categorical variables #####################
    all_cat_vars = ['timesofday','dayofweek','holiday','peak','hour','working_day','season','month']
    cat_vars = [categorical_var for categorical_var in all_cat_vars if categorical_var in predictors]

    # categorical variables
    cat_pipe = Pipeline([
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])

    cat_encoder = 'cat', cat_pipe, cat_vars

    ##################### Numerical variables #####################
    all_num_vars = ['rain', 'temp', 'rhum','wdsp','temp_r']
    num_vars = [numerical_var for numerical_var in all_num_vars if numerical_var in predictors]

    num_pipe = Pipeline([
        ('scaler', StandardScaler())
        # ('scaler', MinMaxScaler())
    ])

    num_enconder =  'num', num_pipe, num_vars

    ##################### Ordinal variables #####################
    all_ord_vars = ['wind_speed_group','rainfall_intensity']
    ord_vars = [ordinal_var for ordinal_var in all_ord_vars if ordinal_var in predictors]

    ordinal_cols_mapping = []
    if 'wind_speed_group' in predictors:
        ordinal_cols_mapping.append(
            {"col":"wind_speed_group",    
            "mapping": {
                'Calm / Light Breeze': 0, 
                'Breeze': 1, 
                'Moderate Breeze': 2, 
                'Strong Breeze / Near Gale': 3, 
                'Gale / Storm': 4
            }}
        )

    if 'rainfall_intensity' in predictors:
        ordinal_cols_mapping.append(
            {"col":"rainfall_intensity",    
            "mapping": {
                'no rain': 0, 
                'drizzle': 1, 
                'light rain': 2, 
                'moderate rain': 3, 
                'heavy rain': 4
            }}
        )

    # ordinal variables
    ord_pipe = Pipeline([
        ('ordinal', ce.OrdinalEncoder(mapping=ordinal_cols_mapping))
    ])

    ord_enconder =  'ordinal', ord_pipe, ord_vars
    
    #################################################################################
    
    orig_vars = [var for var in predictors if var not in cat_vars and var not in num_vars and var not in ord_vars]
    orig_enconder = 'pass_vars', 'passthrough', orig_vars
     # ['temp_bin','rhum_bin']
    # ord_pipe = 'passthrough'

    transformers_list = []
    transformers_list.append(cat_encoder) if cat_vars else None
    transformers_list.append(ord_enconder) if ord_vars else None
    transformers_list.append(num_enconder) if num_vars else None
    # transformers_list.append(orig_enconder) if orig_vars else None
    
    return ColumnTransformer(transformers=transformers_list, 
                             remainder='drop')

In [9]:
def summarize_dict(dictionary, function):
    return {k: function(v) for k,v in dictionary.items()}

In [10]:
def kfold_score(params, predictors, X=X, y=y, n_folds=5, verbose=50, early_stopping_rounds=10):
    
    pipe_xgboost = Pipeline([
        ('preprocessor', preprocessor(predictors)),
        ('model', xgb.XGBRegressor(**params))
    ])

    X = X[[c for c in X.columns if c in predictors]]
    cv = KFold(n_splits=n_folds, shuffle=True, random_state=2022)
    scores = {"train_rsme":[],"val_rsme":[],"train_mae":[],"val_mae":[]}

    for n_fold, (train_index, test_index) in enumerate(cv.split(X, y)):
        print('#'*40, f'Fold {n_fold+1} out of {cv.n_splits}', '#'*40)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Xy = xgb.DMatrix(X_train, y_train, enable_categorical=True)
        X_test_transformed = pipe_xgboost['preprocessor'].fit_transform(X_test)
        pipe_xgboost.fit(X_train, y_train,
                         model__eval_set=[(X_test_transformed, y_test)], 
                         model__early_stopping_rounds=early_stopping_rounds,
                         model__verbose=verbose)
        # pipe_xgboost.fit(X_train, y_train)
        # print(pipe_xgboost['model'].evals_result())

        # Predict on training and validation set
        y_pred_train = pipe_xgboost.predict(X_train)
        y_pred_val = pipe_xgboost.predict(X_test)

        # Calculate the RSME and MAE
        # If squared = True returns MSE value, if False returns RMSE value.
        scores['train_rsme'].append(metrics.mean_squared_error(y_train, y_pred_train, squared=False))
        scores['val_rsme'].append(metrics.mean_squared_error(y_test, y_pred_val, squared=False))
        scores['train_mae'].append(metrics.mean_absolute_error(y_train, y_pred_train))
        scores['val_mae'].append(metrics.mean_absolute_error(y_test, y_pred_val))

        print(f"Fold {n_fold+1} - best iteration: {pipe_xgboost['model'].get_booster().best_iteration}\n")

    return summarize_dict(scores, np.mean), pipe_xgboost

In [11]:
#predictors = ['temp','rhum','dayofweek', 'holiday','timesofday','wdsp','rainfall_intensity','peak','working_day', 'hour', 'season']

## Hyperparameters tuning

In [12]:
# Baseline model
predictors = ['temp','rhum','dayofweek', 'holiday','timesofday','wdsp','rainfall_intensity','peak','working_day', 'hour', 'season']

params_xgboost = {'max_depth':3,
                  'n_estimators': 500,
                   'seed': 42,
                   'eval_metric': 'rmse'
                   }

dict_scores, xgb_model = kfold_score(params_xgboost, predictors, n_folds=3)
exp_xgboost = et.Experiment('XGBoost (Baseline)', predictors=predictors, hyperparameters=xgb_model['model'].get_params(),
                               score=get_metrics_to_Experiment(dict_scores), notes='Baseline XGBoost')
experiment_tracker.add_experiment(exp_xgboost)

######################################## Fold 1 out of 3 ########################################
[0]	validation_0-rmse:4.21537
[50]	validation_0-rmse:2.74378
[59]	validation_0-rmse:2.74717
Fold 1 - best iteration: 50

######################################## Fold 2 out of 3 ########################################
[0]	validation_0-rmse:3.78092
[50]	validation_0-rmse:2.50728
[53]	validation_0-rmse:2.51117
Fold 2 - best iteration: 43

######################################## Fold 3 out of 3 ########################################
[0]	validation_0-rmse:3.96064
[50]	validation_0-rmse:2.65082
[60]	validation_0-rmse:2.65222
Fold 3 - best iteration: 50

--- New Experiment added! ---
ID#: 5551432752 
Algorithm: XGBoost (Baseline) 
Predictors: ['temp', 'rhum', 'dayofweek', 'holiday', 'timesofday', 'wdsp', 'rainfall_intensity', 'peak', 'working_day', 'hour', 'season']
Hyperparameters: {'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_b

In [13]:
predictors = ['temp','rhum','dayofweek', 'holiday','timesofday','wdsp','rainfall_intensity','peak','working_day', 'hour', 'season']

params_xgboost = {'max_depth':3,
                  'eta': 0.2,
                  'n_estimators': 500,
                  'subsample': 1,
                  'colsample_bytree': 0.5,
                  'gamma': 1,
                   'seed': 42,
                   'eval_metric': 'rmse'
                   }

dict_scores, xgb_model = kfold_score(params_xgboost, predictors, n_folds=3)
exp_xgboost = et.Experiment('XGBoost 1', predictors=predictors, hyperparameters=xgb_model['model'].get_params(),
                               score=get_metrics_to_Experiment(dict_scores), notes='')
experiment_tracker.add_experiment(exp_xgboost)

######################################## Fold 1 out of 3 ########################################
[0]	validation_0-rmse:4.51870
[50]	validation_0-rmse:2.74515
[81]	validation_0-rmse:2.73040
Fold 1 - best iteration: 71

######################################## Fold 2 out of 3 ########################################
[0]	validation_0-rmse:4.08433
[50]	validation_0-rmse:2.50050
[87]	validation_0-rmse:2.48756
Fold 2 - best iteration: 78

######################################## Fold 3 out of 3 ########################################
[0]	validation_0-rmse:4.26917
[50]	validation_0-rmse:2.66483
[64]	validation_0-rmse:2.66475
Fold 3 - best iteration: 54

--- New Experiment added! ---
ID#: 5551376896 
Algorithm: XGBoost 1 
Predictors: ['temp', 'rhum', 'dayofweek', 'holiday', 'timesofday', 'wdsp', 'rainfall_intensity', 'peak', 'working_day', 'hour', 'season']
Hyperparameters: {'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1

In [14]:
predictors = ['temp','rhum','dayofweek', 'holiday','timesofday','wdsp','rainfall_intensity','peak','working_day', 'hour', 'season']

params_xgboost = {'max_depth':9,
                  'eta': 0.01,
                  'n_estimators': 1000,
                  'subsample': 0.7,
                  'colsample_bytree': 0.5,
                  'gamma': 1,
                   'seed': 42,
                   'eval_metric': 'rmse'
                   }

dict_scores, xgb_model = kfold_score(params_xgboost, predictors, n_folds=5, verbose=250, early_stopping_rounds=30)
exp_xgboost = et.Experiment('XGBoost 2', predictors=predictors, hyperparameters=xgb_model['model'].get_params(), 
                            score=get_metrics_to_Experiment(dict_scores))
experiment_tracker.add_experiment(exp_xgboost)

######################################## Fold 1 out of 5 ########################################
[0]	validation_0-rmse:4.98932
[250]	validation_0-rmse:2.74287
[464]	validation_0-rmse:2.69763
Fold 1 - best iteration: 434

######################################## Fold 2 out of 5 ########################################
[0]	validation_0-rmse:4.92424
[250]	validation_0-rmse:2.67194
[500]	validation_0-rmse:2.62396
[555]	validation_0-rmse:2.62402
Fold 2 - best iteration: 525

######################################## Fold 3 out of 5 ########################################
[0]	validation_0-rmse:4.69002
[250]	validation_0-rmse:2.47367
[485]	validation_0-rmse:2.43120
Fold 3 - best iteration: 456

######################################## Fold 4 out of 5 ########################################
[0]	validation_0-rmse:4.71785
[250]	validation_0-rmse:2.55160
[460]	validation_0-rmse:2.51855
Fold 4 - best iteration: 430

######################################## Fold 5 out of 5 #######################

In [15]:
predictors = ['temp','rhum','dayofweek', 'holiday','timesofday','wdsp','rainfall_intensity','peak','working_day', 'hour', 'season']

params_xgboost = {'max_depth':9,
                  'eta': 0.01,
                  'n_estimators': 1000,
                  'subsample': 0.7,
                  'colsample_bytree': 0.5,
                  'gamma': 1.5,
                   'seed': 42,
                   'eval_metric': 'rmse'
                   }

dict_scores, xgb_model = kfold_score(params_xgboost, predictors, n_folds=5, verbose=250, early_stopping_rounds=30)
exp_xgboost = et.Experiment('XGBoost (gamma: 1.5)', predictors=predictors, hyperparameters=xgb_model['model'].get_params(), 
                            score=get_metrics_to_Experiment(dict_scores))
experiment_tracker.add_experiment(exp_xgboost)

######################################## Fold 1 out of 5 ########################################
[0]	validation_0-rmse:4.98932
[250]	validation_0-rmse:2.74273
[465]	validation_0-rmse:2.69698
Fold 1 - best iteration: 435

######################################## Fold 2 out of 5 ########################################
[0]	validation_0-rmse:4.92424
[250]	validation_0-rmse:2.67145
[500]	validation_0-rmse:2.62315
[562]	validation_0-rmse:2.62348
Fold 2 - best iteration: 533

######################################## Fold 3 out of 5 ########################################
[0]	validation_0-rmse:4.69001
[250]	validation_0-rmse:2.47431
[500]	validation_0-rmse:2.43228
Fold 3 - best iteration: 470

######################################## Fold 4 out of 5 ########################################
[0]	validation_0-rmse:4.71784
[250]	validation_0-rmse:2.54976
[456]	validation_0-rmse:2.51627
Fold 4 - best iteration: 426

######################################## Fold 5 out of 5 #######################

In [16]:
predictors = ['temp','rhum','dayofweek', 'holiday','timesofday','wdsp','rainfall_intensity','peak','working_day', 'hour', 'season']

params_xgboost = {'max_depth':11,
                  'eta': 0.001,
                  'n_estimators': 5000,
                  'subsample': 0.7,
                  'colsample_bytree': 0.5,
                  'gamma': 1.5,
                   'seed': 42,
                   'eval_metric': 'rmse'
                   }

dict_scores, xgb_model = kfold_score(params_xgboost, predictors, n_folds=5, verbose=250, early_stopping_rounds=30)
exp_xgboost = et.Experiment('XGBoost (eta: 0.001)', predictors=predictors, hyperparameters=xgb_model['model'].get_params(), 
                            score=get_metrics_to_Experiment(dict_scores))
experiment_tracker.add_experiment(exp_xgboost)

######################################## Fold 1 out of 5 ########################################
[0]	validation_0-rmse:5.01778
[250]	validation_0-rmse:4.34078
[500]	validation_0-rmse:3.84679
[750]	validation_0-rmse:3.49268
[1000]	validation_0-rmse:3.24291
[1250]	validation_0-rmse:3.06872
[1500]	validation_0-rmse:2.94775
[1750]	validation_0-rmse:2.86530
[2000]	validation_0-rmse:2.80914
[2250]	validation_0-rmse:2.77180
[2500]	validation_0-rmse:2.74582
[2750]	validation_0-rmse:2.72823
[3000]	validation_0-rmse:2.71654
[3250]	validation_0-rmse:2.70893
[3500]	validation_0-rmse:2.70418
[3750]	validation_0-rmse:2.70196
[3969]	validation_0-rmse:2.70103
Fold 1 - best iteration: 3940

######################################## Fold 2 out of 5 ########################################
[0]	validation_0-rmse:4.95270
[250]	validation_0-rmse:4.27162
[500]	validation_0-rmse:3.77503
[750]	validation_0-rmse:3.41854
[1000]	validation_0-rmse:3.16629
[1250]	validation_0-rmse:2.99146
[1500]	validation_0-rmse:2

In [17]:
predictors = ['temp','rhum','dayofweek', 'holiday','timesofday','wdsp','rainfall_intensity','peak','working_day', 'hour', 'season']

params_xgboost = {'max_depth':9,
                  'eta': 0.001,
                  'n_estimators': 5000,
                  'subsample': 0.8,
                  'colsample_bytree': 0.5,
                  'gamma': 1.5,
                   'seed': 42,
                   'eval_metric': 'rmse'
                   }

dict_scores, xgb_model = kfold_score(params_xgboost, predictors, n_folds=5, verbose=250, early_stopping_rounds=30)
exp_xgboost = et.Experiment('XGBoost (max_depth: 9 and subsample: 0.8)', predictors=predictors, hyperparameters=xgb_model['model'].get_params(), 
                            score=get_metrics_to_Experiment(dict_scores))
experiment_tracker.add_experiment(exp_xgboost)

######################################## Fold 1 out of 5 ########################################
[0]	validation_0-rmse:5.01778
[250]	validation_0-rmse:4.33523
[500]	validation_0-rmse:3.83885
[750]	validation_0-rmse:3.48379
[1000]	validation_0-rmse:3.23314
[1250]	validation_0-rmse:3.05909
[1500]	validation_0-rmse:2.93807
[1750]	validation_0-rmse:2.85623
[2000]	validation_0-rmse:2.80124
[2250]	validation_0-rmse:2.76408
[2500]	validation_0-rmse:2.73825
[2750]	validation_0-rmse:2.72036
[3000]	validation_0-rmse:2.70884
[3250]	validation_0-rmse:2.70071
[3500]	validation_0-rmse:2.69512
[3750]	validation_0-rmse:2.69129
[4000]	validation_0-rmse:2.68937
[4201]	validation_0-rmse:2.68883
Fold 1 - best iteration: 4172

######################################## Fold 2 out of 5 ########################################
[0]	validation_0-rmse:4.95269
[250]	validation_0-rmse:4.26762
[500]	validation_0-rmse:3.76792
[750]	validation_0-rmse:3.41026
[1000]	validation_0-rmse:3.15849
[1250]	validation_0-rmse:2

In [23]:
def normalized_rsme(value, dataset) -> float:
    return value / max(dataset['count']) - min(dataset['count'])

In [24]:
# Normalized RSME - Root Mean Square Error
normtrain_rsme = normalized_rsme(dict_scores['train_rsme'], df_train)
normtrain_rsme

0.0734499192450501

In [25]:
normtest_rsme = normalized_rsme(dict_scores['val_rsme'], df_test)
normtest_rsme

0.13570718936061674

In [21]:
joblib.dump(xgb_model['model'], '../models/XGBoost.pkl')

['../models/XGBoost.pkl']

In [26]:
experiment_tracker.to_excel('../documentation/experiment_tracker_xgboost.xlsx')

In [27]:
%reload_ext watermark
%watermark -a "Leandro Pessini" -n -u -v -iv -w

Author: Leandro Pessini

Last updated: Sat Jul 02 2022

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 8.3.0

pandas           : 1.3.0
sys              : 3.9.6 | packaged by conda-forge | (default, Jul 11 2021, 03:36:15) 
[Clang 11.1.0 ]
matplotlib       : 3.4.2
category_encoders: 2.4.0
xgboost          : 1.4.0
seaborn          : 0.11.1
joblib           : 1.0.1
numpy            : 1.21.1
statsmodels      : 0.13.2
sklearn          : 1.0.2

Watermark: 2.3.0



<!-- > https://pierpaolo28.github.io/blog/blog25/#bayesian-optimization

> https://www.kdnuggets.com/2019/07/xgboost-random-forest-bayesian-optimisation.html

> https://www.kaggle.com/code/neerajmohan/randomforest-model-with-bayesian-optimization/notebook

> https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/ -->

<img title="GitHub Mark" src="./img/GitHub-Mark-64px.png" style="height: 32px; padding-right: 15px" alt="GitHub Mark" align="left"> [GitHub repository](https://github.com/pessini/moby-bikes) <br>Author: Leandro Pessini